In [ ]:
# Getting the data generators for the training and validation sets
import os
import sys

module_path = os.path.abspath('../utils')
print("Adding to sys.path:", module_path)
sys.path.append(module_path)

from data_prep import get_data_generators
train_gen, valid_gen, full_gen, _ = get_data_generators('../data/pokemon.csv', '../data/pokemon-img/pokemon/pokemon/')

In [ ]:
import matplotlib.pyplot as plt
x, y = next(train_gen)
print(x.shape, y.shape)

plt.imshow(x[2])
plt.show()

In [6]:
from tensorflow.keras.applications import EfficientNetB0

base_model = EfficientNetB0(
    include_top=False,
    weights='imagenet',
    input_shape=(224, 224, 3)
)

# defining the model based on the base model
from tensorflow.keras import layers, models
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import BinaryCrossentropy
from tensorflow.keras.metrics import Precision,Recall, AUC

# freeze the base
base_model.trainable = False

model = models.Sequential([
    base_model,
    layers.GlobalAveragePooling2D(),
    layers.Dropout(0.3),
    layers.Dense(128, activation='relu'),
    layers.Dropout(0.3),
    layers.Dense(18, activation='sigmoid')  # 18 is the number of pokemon types so 18 classes
])

model.compile(
    optimizer=Adam(1e-4),
    loss=BinaryCrossentropy(),
    metrics=[AUC(name='auc', multi_label=True), Precision(name='precision'), Recall(name='recall')]

)
model.summary()



Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 efficientnetb0 (Functional  (None, 7, 7, 1280)        4049571   
 )                                                               
                                                                 
 global_average_pooling2d (  (None, 1280)              0         
 GlobalAveragePooling2D)                                         
                                                                 
 dropout (Dropout)           (None, 1280)              0         
                                                                 
 dense (Dense)               (None, 128)               163968    
                                                                 
 dropout_1 (Dropout)         (None, 128)               0         
                                                                 
 dense_1 (Dense)             (None, 18)               

In [7]:
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
history = model.fit(
    train_gen,
    validation_data=valid_gen,
    epochs=50,  # longer, EarlyStopping will stop if plateau
    callbacks=[EarlyStopping(
        monitor='val_loss',
        patience=15,       # stop if no improvement for 5 epochs
        restore_best_weights=True
    ), ReduceLROnPlateau(
        monitor='val_loss',
        factor=0.5,       # reduce LR by half
        patience=3,
        min_lr=1e-6
    )]
)

Epoch 1/50

18/18 [==============================] - 15s 586ms/step - loss: 0.6221 - auc: 0.4937 - precision: 0.0670 - recall: 0.2007 - val_loss: 0.5396 - val_auc: 0.5138 - val_precision: 0.0000e+00 - val_recall: 0.0000e+00 - lr: 1.0000e-04
Epoch 2/50
18/18 [==============================] - 11s 604ms/step - loss: 0.5028 - auc: 0.5018 - precision: 0.0770 - recall: 0.0554 - val_loss: 0.4367 - val_auc: 0.5003 - val_precision: 0.0000e+00 - val_recall: 0.0000e+00 - lr: 1.0000e-04
Epoch 3/50
18/18 [==============================] - 12s 670ms/step - loss: 0.4218 - auc: 0.4974 - precision: 0.0742 - recall: 0.0196 - val_loss: 0.3637 - val_auc: 0.5047 - val_precision: 0.0000e+00 - val_recall: 0.0000e+00 - lr: 1.0000e-04
Epoch 4/50
18/18 [==============================] - 12s 676ms/step - loss: 0.3664 - auc: 0.5095 - precision: 0.0678 - recall: 0.0092 - val_loss: 0.3178 - val_auc: 0.4889 - val_precision: 0.0000e+00 - val_recall: 0.0000e+00 - lr: 1.0000e-04
Epoch 5/50
18/18 [=====================